## Runway database update
This notebook guides you through the process of adding or updating runways in the database of airports.

In [1]:
import json
import numpy as np
from src.ges.ges_dataset import add_or_update_runways

### Open and visualize the database
The database is stored in the `data/` folder, but you can use any json file if needed. Note that this database `.json` file is reused during the labeling process and must remain consistent between the generation of a scenario and the labeling of the resulting images.

In [25]:
data_file = 'data/runways_database.json'
ICAO_airport_list = []
with open(data_file, 'r') as f:
    runways_database = json.load(f)
ICAO_airport_list = [airport for airport in runways_database]

print(ICAO_airport_list)

['ZBAA', 'LTAI', 'LICJ', 'LPPT', 'LIRN', 'EDDV', 'LSZH', 'LEMD', 'LWSK', 'VHHH', 'CYUL', 'VRMM', 'LFSB', 'LCPH', 'GCRR', 'EHAM', 'SAEZ', 'LFRN', 'LFRS', 'LFPO', 'LFQQ', 'LFST', 'LFMP', 'DAAG', 'VABB', 'BIRK', 'YBBN', 'CYVR', 'CYYZ', 'KIAH', 'KJFK', 'KMIA', 'KSFO', 'MDSD', 'RJAA', 'RJTT', 'WSSS', 'FMEP', 'VQPR', 'DIAP', 'KMSY', 'SRLI', 'VDPP', 'RPMD', 'VOTV', 'YMLT', 'OMAD', 'HTDA', 'FTTJ', 'LOWL', 'LGSM', 'DAAS', 'SEQM']


In [19]:
# You can directly visualize the airport database in the .json file, 
# or you can explore the runways of a specific airport as follows
data_file = 'data/runways_database.json'
airport = 'LTAI' # Replace with the chosen airport ICAO code

with open(data_file, 'r') as f:
    runways_database = json.load(f)
    for runway in runways_database[airport]:
        print("Runway", runway)
        for point in runways_database[airport][runway]:
            print("| Corner ", point)
            print("| |  latitude: ", runways_database[airport][runway][point]['coordinate']['latitude'])
            print("| | longitude: ", runways_database[airport][runway][point]['coordinate']['longitude'])
            print("| |  altitude: ", runways_database[airport][runway][point]['coordinate']['altitude'])

Runway 36R
| Corner  A
| |  latitude:  36.911331891325375
| | longitude:  30.81166080286701
| |  altitude:  52.0
| Corner  B
| |  latitude:  36.91137639182183
| | longitude:  30.811169288076773
| |  altitude:  52.0
| Corner  C
| |  latitude:  36.880923205425624
| | longitude:  30.806972063987775
| |  altitude:  47.0
| Corner  D
| |  latitude:  36.88088029626469
| | longitude:  30.807466260961807
| |  altitude:  47.0
Runway 18L
| Corner  A
| |  latitude:  36.880923205425624
| | longitude:  30.806972063987775
| |  altitude:  47.0
| Corner  B
| |  latitude:  36.88088029626469
| | longitude:  30.807466260961807
| |  altitude:  47.0
| Corner  C
| |  latitude:  36.911331891325375
| | longitude:  30.81166080286701
| |  altitude:  52.0
| Corner  D
| |  latitude:  36.91137639182183
| | longitude:  30.811169288076773
| |  altitude:  52.0


# Updating the database
These examples guides you through adding new runways or updating existing ones in the runways database.
- ❗️ By default, a runway corresponds to 2 runway ids, one for each side of the runway. Therefore if 2 `runway_ids` are provided, the 4 coordinates will be stored twice in the database, once for each side of the runway with the proper order. Otherwise, the 4 coordinates will be stored only once for the specific `runway_id` specified.
- ❗️ The order of the points is important: The first 2 points **MUST** correspond to the first runway indicated in the `runway_ids`. The other 2 points **must always be present**, and will correspond to the second runway ID if present. 
- ❗️ Runway IDs are strings and may contain letters. It also implies that the runway `01` and the runway `1` will be stored separately in the json file.   

In [21]:
# Common usage: adding a runway to the database
data_file = 'data/runways_database.json'

airport_name = 'FMEP' # Existing or new airport ICAO code

runway_ids = ['15', '33'] # Supports 1 or 2 runway ID

# Coordinates must be a numpy array of 4 coordinates in WGS84 format:
# ==> Latitude, Longitude, Altitude in meters.
coordinates = np.array([
    [-21.313539792048655, 55.414677867454635, 17 ], # Points 1 and 2 should correspond to the first runway ID
    [-21.313233693596825, 55.414951452761926, 17 ],
    [-21.324679789297647, 55.43004674957136,  14  ], # Points 3 and 4 should correspond to the second runway ID if present,
    [-21.32499113834105,  55.42977957629199,  14  ]  # but these last two points are always required nonetheless
    ])

# This function will add the existing runway to the database or update its coordinates if it exists
add_or_update_runways(data_file, airport_name, runway_ids, coordinates)

## Advanced usage
- Using another database file (❗️ the file must exist already)

In [23]:
# You can specify another file if needed
runways_db_name = 'data/My_runway_database.json' # (file must exist!)
airport_name = 'YBBN'
runways = ['01', '19'] # Supports 1 or 2 runway ID.
# Pay attention to the '01' here, as it might be redundant with an existing runway '1'
coordinates = np.array([
    [-27.402915287771446, 153.11842342220433, -1 ], # Points 1 and 2 should correspond to the first runway ID
    [-27.402734367704532, 153.11802790132325, -1 ],
    [-27.374768439323887, 153.1345185366854,  -3 ], # Points 3 and 4 should correspond to the second runway ID if present
    [-27.37459000204882,  153.13412570163402, -3 ]])

add_or_update_runways(runways_db_name, airport_name, runways, coordinates)

- Building a database of buildings or any element visible in aerial images (❗️ file must exist)

In [24]:
# We could even build another type of database, for instance specifying the coordinates of buildings
buildings_db_name = 'data/buildings_database.json' # (file must exist!)
building_code = 'IRT'
building = ['SaintEx'] # You can specify a single element, still with its 4 coordinates
coordinates = np.array([[43.56373298797086, 1.4883420029868122, 160],
                        [43.56329195043376, 1.488868309946021,  160],
                        [43.56290020261955, 1.4882632359589036, 160],
                        [43.56332827129408, 1.487761991235848,  160]])

add_or_update_runways(buildings_db_name, building_code, building, coordinates)